<a href="https://colab.research.google.com/github/Lococo231/MO_Tasks/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 5.2 - Решение

По итоговому датасету:

1. Преобразовать текстовые данные (очистка + токенизация + нормализация).
2. Обучить модель **Word2Vec** на текстах.
3. Заполнить пропуски в числовых признаках через **SimpleImputer**.
4. Выбрать алгоритм **кластеризации** и построить кластеры.
5. Построить модель для **предсказания кластера** (supervised) с подбором гиперпараметров через **GridSearchCV**.


## Установка библиотек

In [ ]:
!pip install pymorphy3 gensim scikit-learn nltk

## Импорт библиотек

In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer

import re

# Определяем русские стоп-слова
RUSSIAN_STOPWORDS = {
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все',
    'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по',
    'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему',
    'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть',
    'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом',
    'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для',
    'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз',
    'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому',
    'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем',
    'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно',
    'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот',
    'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три',
    'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть'
}

print("Библиотеки импортированы!")

Библиотеки импортированы!


## 1. Загрузка данных из базы данных

In [ ]:
# Укажите путь к вашей базе данных
DB_FILE = "/content/news_database_final.db"  # Измените на путь к вашему файлу

# Открываем соединение с SQLite
conn = sqlite3.connect(DB_FILE)

# Читаем таблицу в DataFrame
df = pd.read_sql_query("SELECT * FROM news_articles;", conn)

# Закрываем соединение
conn.close()

print(f"Загружено строк: {len(df)}")
print(f"Колонок: {len(df.columns)}")
print(f"\nКолонки: {df.columns.tolist()}")

DatabaseError: Execution failed on sql 'SELECT * FROM news_articles;': database disk image is malformed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.head()

In [ ]:
df.info()

## 2. Предобработка текстовых данных

In [ ]:
TOKEN_RE = re.compile(r"[А-Яа-яA-Za-z]+", flags=re.U)

def preprocess_text(text: str):
    """
    Упрощенная предобработка текста:
    - Приведение к нижнему регистру
    - Токенизация
    - Удаление стоп-слов и коротких токенов
    """
    if not isinstance(text, str):
        text = "" if text is None else str(text)

    # Приведение к нижнему регистру
    text = text.lower()

    # Токенизация
    tokens = TOKEN_RE.findall(text)

    result = []
    for token in tokens:
        # Отсекаем короткие токены
        if len(token) <= 2:
            continue

        # Проверка на стоп-слова для русских слов
        if re.match(r"[а-я]", token):
            if token in RUSSIAN_STOPWORDS:
                continue

        result.append(token)

    return result

# Применяем предобработку
text_col = "description"
tokenized_texts = df[text_col].astype(str).apply(preprocess_text).tolist()

print(f"Всего документов: {len(tokenized_texts)}")
print(f"Пример токенизированного текста (первые 20 слов):")
print(tokenized_texts[0][:20])

## 3. Векторизация текстов (TF-IDF вместо Word2Vec)

In [ ]:
# Объединяем токены обратно в строки для TF-IDF
texts_for_tfidf = [' '.join(tokens) for tokens in tokenized_texts]

# Создаем TF-IDF векторы
tfidf = TfidfVectorizer(max_features=100)
doc_vectors = tfidf.fit_transform(texts_for_tfidf).toarray()

print(f"Размерность матрицы векторов документов: {doc_vectors.shape}")

## 4. Обработка числовых признаков (SimpleImputer)

In [ ]:
num_cols = ["comments_count", "rating"]
numeric_features = df[num_cols].copy()

print(f"Пропуски в числовых признаках:")
print(numeric_features.isnull().sum())

# Импьютация пропусков
imputer = SimpleImputer(strategy="median")
numeric_imputed = imputer.fit_transform(numeric_features)

print(f"\nПосле импьютации:")
print(f"  Размерность: {numeric_imputed.shape}")
print(f"  Пропуски: {np.isnan(numeric_imputed).sum()}")

## 5. Кластеризация

In [ ]:
# Снижение размерности для кластеризации
pca = PCA(n_components=50, random_state=42)
doc_vectors_pca = pca.fit_transform(doc_vectors)
print(f"Объясненная дисперсия (первые 50 компонент): {pca.explained_variance_ratio_.sum():.3f}")

# Подготовка данных для кластеризации
X_clustering = np.hstack([doc_vectors_pca, numeric_imputed])
print(f"Размерность данных для кластеризации: {X_clustering.shape}")

In [ ]:
# Кластеризация методом KMeans
optimal_k = 5
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_clustering)

# Добавляем метки кластеров в датафрейм
df["cluster"] = cluster_labels

print(f"\nРезультаты KMeans (k={optimal_k}):")
unique_clusters, counts = np.unique(cluster_labels, return_counts=True)
for cluster_id, count in zip(unique_clusters, counts):
    print(f"  Кластер {cluster_id}: {count} документов")

In [ ]:
# Визуализация кластеров
pca_2d = PCA(n_components=2, random_state=42)
doc_vectors_2d = pca_2d.fit_transform(doc_vectors)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(
    doc_vectors_2d[:, 0],
    doc_vectors_2d[:, 1],
    c=cluster_labels,
    cmap='tab10',
    s=30,
    alpha=0.6
)
plt.colorbar(scatter, label='Cluster ID')
plt.title('Визуализация кластеров документов (PCA 2D)', fontsize=14)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Примеры документов из каждого кластера
print("Примеры документов из каждого кластера:\n")
for cluster_id in range(optimal_k):
    print(f"--- Кластер {cluster_id} ---")
    cluster_docs = df[df["cluster"] == cluster_id]["title"].head(3)
    for i, title in enumerate(cluster_docs, 1):
        print(f"  {i}. {title}")
    print()

## 6. Подготовка данных для Supervised Learning

In [ ]:
# Объединяем все признаки
X_features = np.hstack([doc_vectors, numeric_imputed])
y = df["cluster"].astype(int)

print(f"Размерность матрицы признаков X: {X_features.shape}")
print(f"Размерность целевой переменной y: {y.shape}")
print(f"\nРаспределение классов:")
print(y.value_counts().sort_index())

# Разбиение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print(f"\nРазмеры выборок:")
print(f"  Train: {X_train.shape}")
print(f"  Test: {X_test.shape}")

## 7. Модель 1: Decision Tree с GridSearchCV

In [ ]:
dt = DecisionTreeClassifier(random_state=42)

param_grid_dt = {
    "max_depth": [None, 5, 10, 15, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "criterion": ["gini", "entropy"],
}

grid_dt = GridSearchCV(
    estimator=dt,
    param_grid=param_grid_dt,
    scoring="accuracy",
    cv=3,
    n_jobs=-1,
)

grid_dt.fit(X_train, y_train)

print(f"Лучшие параметры: {grid_dt.best_params_}")
print(f"Лучший CV accuracy: {grid_dt.best_score_:.4f}")

# Оценка на тестовой выборке
best_dt = grid_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test)

print(f"\n--- РЕЗУЛЬТАТЫ DECISION TREE ---")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_dt))

## 8. Модель 2: KNN с GridSearchCV

In [ ]:
knn = KNeighborsClassifier()

param_grid_knn = {
    "n_neighbors": [3, 5, 7, 10, 15],
    "weights": ["uniform", "distance"],
    "metric": ["cosine", "euclidean"],
}

grid_knn = GridSearchCV(
    estimator=knn,
    param_grid=param_grid_knn,
    scoring="accuracy",
    cv=3,
    n_jobs=-1,
)

grid_knn.fit(X_train, y_train)

print(f"Лучшие параметры: {grid_knn.best_params_}")
print(f"Лучший CV accuracy: {grid_knn.best_score_:.4f}")

# Оценка на тестовой выборке
best_knn = grid_knn.best_estimator_
y_pred_knn = best_knn.predict(X_test)

print(f"\n--- РЕЗУЛЬТАТЫ KNN ---")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_knn))

## 9. Сравнение моделей

In [ ]:
results = pd.DataFrame({
    'Модель': ['Decision Tree', 'KNN'],
    'CV Accuracy': [grid_dt.best_score_, grid_knn.best_score_],
    'Test Accuracy': [
        accuracy_score(y_test, y_pred_dt),
        accuracy_score(y_test, y_pred_knn)
    ],
    'Лучшие параметры': [
        str(grid_dt.best_params_),
        str(grid_knn.best_params_)
    ]
})

print(results.to_string(index=False))